In [43]:
import os
import math
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

corpusroot = r'C:\Users\gupil\Downloads\Assignments\papa\P1 (2)\P1\US_Inaugural_Addresses'

# Initialize tokenizer, stemmer, and stopwords
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


In [44]:
# Function to read and preprocess text files

def read_and_preprocess(corpusroot):
    documents = {}
    for filename in os.listdir(corpusroot):
        if filename.startswith(('0', '1', '2', '3')):
            file_path = os.path.join(corpusroot, filename)
            with open(file_path, "r", encoding='windows-1252') as file:
                doc = file.read().lower()
                tokens = tokenizer.tokenize(doc)
                stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
                documents[filename] = {'tokens': tokens, 'stemmed_tokens': stemmed_tokens}
    return documents

# Read and preprocess documents
preprocessed_documents = read_and_preprocess(corpusroot)

# Print tokens and stemmed tokens for each document
for filename, info in preprocessed_documents.items():
    print(f"Tokens for document {filename}:")
    print(info['tokens'])
   


Tokens for document 01_washington_1789.txt:
['george', 'washington', 'fellow', 'citizens', 'of', 'the', 'senate', 'and', 'of', 'the', 'house', 'of', 'representatives', 'i', 'among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no', 'event', 'could', 'have', 'filled', 'me', 'with', 'greater', 'anxieties', 'than', 'that', 'of', 'which', 'the', 'notification', 'was', 'transmitted', 'by', 'your', 'order', 'and', 'received', 'on', 'the', 'th', 'day', 'of', 'the', 'present', 'month', 'on', 'the', 'one', 'hand', 'i', 'was', 'summoned', 'by', 'my', 'country', 'whose', 'voice', 'i', 'can', 'never', 'hear', 'but', 'with', 'veneration', 'and', 'love', 'from', 'a', 'retreat', 'which', 'i', 'had', 'chosen', 'with', 'the', 'fondest', 'predilection', 'and', 'in', 'my', 'flattering', 'hopes', 'with', 'an', 'immutable', 'decision', 'as', 'the', 'asylum', 'of', 'my', 'declining', 'years', 'a', 'retreat', 'which', 'was', 'rendered', 'every', 'day', 'more', 'necessary', 'as', 'well', 'as', 'more', 'd

In [45]:
# Read and preprocess documents
preprocessed_documents = read_and_preprocess(corpusroot)

# Print stemmed tokens for each document
for filename, info in preprocessed_documents.items():
    print(f"Stemmed tokens for document {filename}:")
    print(info['stemmed_tokens'])


Stemmed tokens for document 01_washington_1789.txt:
['georg', 'washington', 'fellow', 'citizen', 'senat', 'hous', 'repres', 'among', 'vicissitud', 'incid', 'life', 'event', 'could', 'fill', 'greater', 'anxieti', 'notif', 'transmit', 'order', 'receiv', 'th', 'day', 'present', 'month', 'one', 'hand', 'summon', 'countri', 'whose', 'voic', 'never', 'hear', 'vener', 'love', 'retreat', 'chosen', 'fondest', 'predilect', 'flatter', 'hope', 'immut', 'decis', 'asylum', 'declin', 'year', 'retreat', 'render', 'everi', 'day', 'necessari', 'well', 'dear', 'addit', 'habit', 'inclin', 'frequent', 'interrupt', 'health', 'gradual', 'wast', 'commit', 'time', 'hand', 'magnitud', 'difficulti', 'trust', 'voic', 'countri', 'call', 'suffici', 'awaken', 'wisest', 'experienc', 'citizen', 'distrust', 'scrutini', 'qualif', 'could', 'overwhelm', 'despond', 'one', 'inherit', 'inferior', 'endow', 'natur', 'unpract', 'duti', 'civil', 'administr', 'ought', 'peculiarli', 'consciou', 'defici', 'conflict', 'emot', 'dare'

In [46]:
# Calculate term frequencies
tf = {filename: {word: token_list['stemmed_tokens'].count(word) for word in set(token_list['stemmed_tokens'])}
      for filename, token_list in preprocessed_documents.items()}

# Calculate document frequencies
df = {}
for token_list in preprocessed_documents.values():
    for word in set(token_list['stemmed_tokens']):
        df[word] = df.get(word, 0) + 1

# Calculate IDF
idf = {word: math.log(len(preprocessed_documents) / df[word]) for word in df}

# Calculate TF-IDF
tfidf = {filename: {word: (1 + math.log(freq)) * idf[word] for word, freq in freq_dict.items()}
         for filename, freq_dict in tf.items()}



In [47]:
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
    sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator



In [48]:
def query(qstring):
    qtokens = tokenizer.tokenize(qstring.lower())
    qtokens = [word for word in qtokens if word not in stop_words]
    qtokens = [stemmer.stem(word) for word in qtokens]

    qtf = {word: qtokens.count(word) for word in set(qtokens)}
    qtfidf = {word: (1 + math.log(freq)) for word, freq in qtf.items()}

    scores = {filename: cosine_similarity(doc_vector, qtfidf) for filename, doc_vector in tfidf.items()}
    best_doc = max(scores, key=scores.get)
    return best_doc, scores[best_doc]



In [49]:
def getidf(token):
    token = stemmer.stem(token.lower())
    return idf.get(token, -1)



In [50]:
def getweight(filename, token):
    token = stemmer.stem(token.lower())
    return tfidf.get(filename, {}).get(token, 0)



In [51]:
# Test cases
print("%.12f" % getidf('children'))
print("%.12f" % getidf('foreign'))
print("%.12f" % getidf('people'))
print("%.12f" % getidf('honor'))
print("%.12f" % getidf('great'))
print("--------------")
print("%.12f" % getweight('19_lincoln_1861.txt', 'constitution'))
print("%.12f" % getweight('23_hayes_1877.txt', 'public'))
print("%.12f" % getweight('25_cleveland_1885.txt', 'citizen'))
print("%.12f" % getweight('09_monroe_1821.txt', 'revenue'))
print("%.12f" % getweight('05_jefferson_1805.txt', 'press'))
print("--------------")
print("(%s, %.12f)" % query("pleasing people"))
print("(%s, %.12f)" % query("war offenses"))
print("(%s, %.12f)" % query("british war"))
print("(%s, %.12f)" % query("texas government"))
print("(%s, %.12f)" % query("cuba government"))

1.321755839982
0.310154928304
0.068992871487
0.182321556794
0.105360515658
--------------
0.476899678927
0.234430551882
0.115193922179
2.216147201398
2.394174164881
--------------
(03_adams_john_1797.txt, 0.061427335145)
(20_lincoln_1865.txt, 0.197066553017)
(07_madison_1813.txt, 0.121179457049)
(15_polk_1845.txt, 0.100658361560)
(29_mckinley_1901.txt, 0.135753933370)
